#**Sentiment Classification Model using Bi-Gram Language Model**

##Necessary Imports

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from nltk import bigrams, FreqDist
import re
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

##File Read

In [ ]:
# Open the file "Movie_Reviews.txt" in read mode and read its content into the variable "movie_reviews"
with open("Movie_Reviews.txt", "r") as file:
    movie_reviews = file.read()

##Seperate Negative and Postive Reviews

In [ ]:
# Split the content based on the headers of positive and negative reviews
positive_reviews_start = movie_reviews.find("Positive Reviews")
negative_reviews_start = movie_reviews.find("Negative Reviews")

In [ ]:
# Extract the positive reviews
positive_reviews = movie_reviews[positive_reviews_start:negative_reviews_start].strip()

# Remove the line after "================"
positive_reviews = positive_reviews.split("\n================")[1]

# Split the positive reviews by newline
positive_reviews_list = positive_reviews.split('\n')

In [ ]:
# Remove the initial header
positive_reviews_list = [review for review in positive_reviews_list if review]

In [ ]:
# Extract the negative reviews
negative_reviews = movie_reviews[negative_reviews_start:].strip()

# Remove the line after "================"
negative_reviews = negative_reviews.split("\n================")[1]

# Split the negative reviews by newline
negative_reviews_list = negative_reviews.split('\n')


In [ ]:
# Remove the initial header
negative_reviews_list = [review for review in negative_reviews_list if review]

In [ ]:
print(positive_reviews_list)

['1. "Forrest Gump is an absolute masterpiece! Tom Hanks delivers an unforgettable performance, and the storytelling is heartwarming. This movie is a journey through life that will make you laugh, cry, and appreciate the simple beauties of existence."', '2. "The Shawshank Redemption is a timeless classic. The powerful themes of hope, friendship, and redemption make it a must-watch. Morgan Freeman and Tim Robbins give exceptional performances in this brilliantly crafted film."', '3. "The epic conclusion to The Lord of the Rings trilogy, The Return of the King, is a cinematic triumph. The breathtaking visuals, epic battles, and emotionally resonant story make it a monumental achievement in filmmaking."', '4. "La La Land is a love letter to the magic of Hollywood and dreams. The chemistry between Ryan Gosling and Emma Stone is enchanting, and the music and dance sequences are a pure delight. A modern musical masterpiece."', '5. "Wes Anderson\'s whimsical style shines in The Grand Budapest

In [ ]:
print(negative_reviews_list)

['1. "The Last Airbender is a disaster of a film adaptation. It butchers the beloved animated series with wooden acting, convoluted storytelling, and cringe-worthy special effects. A letdown for fans and newcomers alike."', '2. "Another Transformers movie, and it\'s just more of the same: mindless explosions, incoherent plotlines, and an overreliance on CGI. This franchise desperately needs an overhaul."', '3. "The Emoji Movie is a blatant cash grab with a shallow, uninspired plot. It fails to deliver clever humor or meaningful messages, making it a forgettable and disappointing animated film."', '4. "Fifty Shades of Grey is a cringe-inducing attempt at romance. Poorly written dialogue and unconvincing chemistry between the leads make it an awkward and unfulfilling cinematic experience."', '5. "Jack and Jill is an unbearable comedy that relies on stale humor and a painfully unfunny portrayal of Adam Sandler in a dual role. It\'s a prime example of lazy filmmaking."', '6. "Superman IV i

##Preprocessing raw text for Sentiment Analysis

###Convert to Lower Case

In [ ]:
# Convert positive reviews to lowercase
positive_reviews_lowercase= [review.lower() for review in positive_reviews_list]
print( positive_reviews_lowercase)

['1. "forrest gump is an absolute masterpiece! tom hanks delivers an unforgettable performance, and the storytelling is heartwarming. this movie is a journey through life that will make you laugh, cry, and appreciate the simple beauties of existence."', '2. "the shawshank redemption is a timeless classic. the powerful themes of hope, friendship, and redemption make it a must-watch. morgan freeman and tim robbins give exceptional performances in this brilliantly crafted film."', '3. "the epic conclusion to the lord of the rings trilogy, the return of the king, is a cinematic triumph. the breathtaking visuals, epic battles, and emotionally resonant story make it a monumental achievement in filmmaking."', '4. "la la land is a love letter to the magic of hollywood and dreams. the chemistry between ryan gosling and emma stone is enchanting, and the music and dance sequences are a pure delight. a modern musical masterpiece."', '5. "wes anderson\'s whimsical style shines in the grand budapest

In [ ]:
# Convert negative reviews to lowercase
negative_reviews_lowercase = [review.lower() for review in negative_reviews_list]
print( negative_reviews_lowercase)

['1. "the last airbender is a disaster of a film adaptation. it butchers the beloved animated series with wooden acting, convoluted storytelling, and cringe-worthy special effects. a letdown for fans and newcomers alike."', '2. "another transformers movie, and it\'s just more of the same: mindless explosions, incoherent plotlines, and an overreliance on cgi. this franchise desperately needs an overhaul."', '3. "the emoji movie is a blatant cash grab with a shallow, uninspired plot. it fails to deliver clever humor or meaningful messages, making it a forgettable and disappointing animated film."', '4. "fifty shades of grey is a cringe-inducing attempt at romance. poorly written dialogue and unconvincing chemistry between the leads make it an awkward and unfulfilling cinematic experience."', '5. "jack and jill is an unbearable comedy that relies on stale humor and a painfully unfunny portrayal of adam sandler in a dual role. it\'s a prime example of lazy filmmaking."', '6. "superman iv i

###Tokenization

In [ ]:
# Tokenize positive and negative reviews into words
tokenized_positive_reviews = [word_tokenize(review) for review in positive_reviews_lowercase]
tokenized_negative_reviews = [word_tokenize(review) for review in negative_reviews_lowercase]

In [ ]:
# Flatten the lists of tokenized words
flatted_positive_reviews = [word for review in tokenized_positive_reviews for word in review]
flatted_negative_reviews = [word for review in tokenized_negative_reviews for word in review]

In [ ]:
# Display the tokenized reviews
print(flatted_positive_reviews)
print(flatted_negative_reviews)

['1', '.', '``', 'forrest', 'gump', 'is', 'an', 'absolute', 'masterpiece', '!', 'tom', 'hanks', 'delivers', 'an', 'unforgettable', 'performance', ',', 'and', 'the', 'storytelling', 'is', 'heartwarming', '.', 'this', 'movie', 'is', 'a', 'journey', 'through', 'life', 'that', 'will', 'make', 'you', 'laugh', ',', 'cry', ',', 'and', 'appreciate', 'the', 'simple', 'beauties', 'of', 'existence', '.', "''", '2', '.', '``', 'the', 'shawshank', 'redemption', 'is', 'a', 'timeless', 'classic', '.', 'the', 'powerful', 'themes', 'of', 'hope', ',', 'friendship', ',', 'and', 'redemption', 'make', 'it', 'a', 'must-watch', '.', 'morgan', 'freeman', 'and', 'tim', 'robbins', 'give', 'exceptional', 'performances', 'in', 'this', 'brilliantly', 'crafted', 'film', '.', "''", '3', '.', '``', 'the', 'epic', 'conclusion', 'to', 'the', 'lord', 'of', 'the', 'rings', 'trilogy', ',', 'the', 'return', 'of', 'the', 'king', ',', 'is', 'a', 'cinematic', 'triumph', '.', 'the', 'breathtaking', 'visuals', ',', 'epic', 'bat

###Remove punctuations

In [ ]:
# Function to remove punctuation
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

In [ ]:
# Remove punctuation from positive reviews
cleaned_positive_reviews = [remove_punctuation(review) for review in flatted_positive_reviews]
print(cleaned_positive_reviews)

['1', '', '', 'forrest', 'gump', 'is', 'an', 'absolute', 'masterpiece', '', 'tom', 'hanks', 'delivers', 'an', 'unforgettable', 'performance', '', 'and', 'the', 'storytelling', 'is', 'heartwarming', '', 'this', 'movie', 'is', 'a', 'journey', 'through', 'life', 'that', 'will', 'make', 'you', 'laugh', '', 'cry', '', 'and', 'appreciate', 'the', 'simple', 'beauties', 'of', 'existence', '', '', '2', '', '', 'the', 'shawshank', 'redemption', 'is', 'a', 'timeless', 'classic', '', 'the', 'powerful', 'themes', 'of', 'hope', '', 'friendship', '', 'and', 'redemption', 'make', 'it', 'a', 'mustwatch', '', 'morgan', 'freeman', 'and', 'tim', 'robbins', 'give', 'exceptional', 'performances', 'in', 'this', 'brilliantly', 'crafted', 'film', '', '', '3', '', '', 'the', 'epic', 'conclusion', 'to', 'the', 'lord', 'of', 'the', 'rings', 'trilogy', '', 'the', 'return', 'of', 'the', 'king', '', 'is', 'a', 'cinematic', 'triumph', '', 'the', 'breathtaking', 'visuals', '', 'epic', 'battles', '', 'and', 'emotionall

In [ ]:
# Remove punctuation from negative reviews
cleaned_negative_reviews = [remove_punctuation(review) for review in flatted_negative_reviews]
print( cleaned_negative_reviews)

['1', '', '', 'the', 'last', 'airbender', 'is', 'a', 'disaster', 'of', 'a', 'film', 'adaptation', '', 'it', 'butchers', 'the', 'beloved', 'animated', 'series', 'with', 'wooden', 'acting', '', 'convoluted', 'storytelling', '', 'and', 'cringeworthy', 'special', 'effects', '', 'a', 'letdown', 'for', 'fans', 'and', 'newcomers', 'alike', '', '', '2', '', '', 'another', 'transformers', 'movie', '', 'and', 'it', 's', 'just', 'more', 'of', 'the', 'same', '', 'mindless', 'explosions', '', 'incoherent', 'plotlines', '', 'and', 'an', 'overreliance', 'on', 'cgi', '', 'this', 'franchise', 'desperately', 'needs', 'an', 'overhaul', '', '', '3', '', '', 'the', 'emoji', 'movie', 'is', 'a', 'blatant', 'cash', 'grab', 'with', 'a', 'shallow', '', 'uninspired', 'plot', '', 'it', 'fails', 'to', 'deliver', 'clever', 'humor', 'or', 'meaningful', 'messages', '', 'making', 'it', 'a', 'forgettable', 'and', 'disappointing', 'animated', 'film', '', '', '4', '', '', 'fifty', 'shades', 'of', 'grey', 'is', 'a', 'crin

###Lemmetization and remove stop words

In [ ]:
# Function for lemmertization and removing stop words
def get_wordnet_pos(tag):
    # Map POS tag to WordNet POS tag
    tag_dict = {"N": "n", "V": "v", "R": "r", "J": "a"}
    return tag_dict.get(tag[0], 'n')  # Default to noun if not found

def lemmatize_reviews(reviews):
    # Perform part-of-speech tagging
    reviews_pos = pos_tag(reviews)

    # Define and remove stop words and numbers
    stop_words = set(stopwords.words("english"))

    reviews_filtered = [word for word, pos in reviews_pos if word not in stop_words and word.isalpha()]

    # Lemmatize the filtered words based on POS
    lemmatizer = WordNetLemmatizer()
    reviews_lemmatized = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos)) for word, pos in pos_tag(reviews_filtered)]

    return reviews_lemmatized

In [ ]:
# Lemmatized and stop word removed positive words
final_positive_words = lemmatize_reviews(cleaned_positive_reviews)
print( final_positive_words)

['forrest', 'gump', 'absolute', 'masterpiece', 'tom', 'hank', 'delivers', 'unforgettable', 'performance', 'storytelling', 'heartwarming', 'movie', 'journey', 'life', 'make', 'laugh', 'cry', 'appreciate', 'simple', 'beauty', 'existence', 'shawshank', 'redemption', 'timeless', 'classic', 'powerful', 'theme', 'hope', 'friendship', 'redemption', 'make', 'mustwatch', 'morgan', 'freeman', 'tim', 'robbins', 'give', 'exceptional', 'performance', 'brilliantly', 'craft', 'film', 'epic', 'conclusion', 'lord', 'ring', 'trilogy', 'return', 'king', 'cinematic', 'triumph', 'breathtaking', 'visuals', 'epic', 'battle', 'emotionally', 'resonant', 'story', 'make', 'monumental', 'achievement', 'filmmaking', 'la', 'la', 'land', 'love', 'letter', 'magic', 'hollywood', 'dream', 'chemistry', 'ryan', 'gosling', 'emma', 'stone', 'enchant', 'music', 'dance', 'sequence', 'pure', 'delight', 'modern', 'musical', 'masterpiece', 'wes', 'anderson', 'whimsical', 'style', 'shine', 'grand', 'budapest', 'hotel', 'quirky',

In [ ]:
# Lemmatized and stop word removed positive words
final_negative_words = lemmatize_reviews(cleaned_negative_reviews)
print( final_negative_words)

['last', 'airbender', 'disaster', 'film', 'adaptation', 'butcher', 'beloved', 'animated', 'series', 'wooden', 'act', 'convolute', 'storytelling', 'cringeworthy', 'special', 'effect', 'letdown', 'fan', 'newcomer', 'alike', 'another', 'transformer', 'movie', 'mindless', 'explosion', 'incoherent', 'plotlines', 'overreliance', 'cgi', 'franchise', 'desperately', 'need', 'overhaul', 'emoji', 'movie', 'blatant', 'cash', 'grab', 'shallow', 'uninspired', 'plot', 'fails', 'deliver', 'clever', 'humor', 'meaningful', 'message', 'make', 'forgettable', 'disappointing', 'animate', 'film', 'fifty', 'shade', 'grey', 'cringeinducing', 'attempt', 'romance', 'poorly', 'write', 'dialogue', 'unconvincing', 'chemistry', 'lead', 'make', 'awkward', 'unfulfilling', 'cinematic', 'experience', 'jack', 'jill', 'unbearable', 'comedy', 'relies', 'stale', 'humor', 'painfully', 'unfunny', 'portrayal', 'adam', 'sandler', 'dual', 'role', 'prime', 'example', 'lazy', 'filmmaking', 'superman', 'iv', 'colossal', 'disappoint

##Implement Bi-Gram Model

In [ ]:
# Generate bigrams for positive and negative reviews
positive_bigrams = list(bigrams(final_positive_words))
negative_bigrams = list(bigrams(final_negative_words))

In [ ]:
print(positive_bigrams)

[('forrest', 'gump'), ('gump', 'absolute'), ('absolute', 'masterpiece'), ('masterpiece', 'tom'), ('tom', 'hank'), ('hank', 'delivers'), ('delivers', 'unforgettable'), ('unforgettable', 'performance'), ('performance', 'storytelling'), ('storytelling', 'heartwarming'), ('heartwarming', 'movie'), ('movie', 'journey'), ('journey', 'life'), ('life', 'make'), ('make', 'laugh'), ('laugh', 'cry'), ('cry', 'appreciate'), ('appreciate', 'simple'), ('simple', 'beauty'), ('beauty', 'existence'), ('existence', 'shawshank'), ('shawshank', 'redemption'), ('redemption', 'timeless'), ('timeless', 'classic'), ('classic', 'powerful'), ('powerful', 'theme'), ('theme', 'hope'), ('hope', 'friendship'), ('friendship', 'redemption'), ('redemption', 'make'), ('make', 'mustwatch'), ('mustwatch', 'morgan'), ('morgan', 'freeman'), ('freeman', 'tim'), ('tim', 'robbins'), ('robbins', 'give'), ('give', 'exceptional'), ('exceptional', 'performance'), ('performance', 'brilliantly'), ('brilliantly', 'craft'), ('craft',

In [ ]:
print(negative_bigrams)

[('last', 'airbender'), ('airbender', 'disaster'), ('disaster', 'film'), ('film', 'adaptation'), ('adaptation', 'butcher'), ('butcher', 'beloved'), ('beloved', 'animated'), ('animated', 'series'), ('series', 'wooden'), ('wooden', 'act'), ('act', 'convolute'), ('convolute', 'storytelling'), ('storytelling', 'cringeworthy'), ('cringeworthy', 'special'), ('special', 'effect'), ('effect', 'letdown'), ('letdown', 'fan'), ('fan', 'newcomer'), ('newcomer', 'alike'), ('alike', 'another'), ('another', 'transformer'), ('transformer', 'movie'), ('movie', 'mindless'), ('mindless', 'explosion'), ('explosion', 'incoherent'), ('incoherent', 'plotlines'), ('plotlines', 'overreliance'), ('overreliance', 'cgi'), ('cgi', 'franchise'), ('franchise', 'desperately'), ('desperately', 'need'), ('need', 'overhaul'), ('overhaul', 'emoji'), ('emoji', 'movie'), ('movie', 'blatant'), ('blatant', 'cash'), ('cash', 'grab'), ('grab', 'shallow'), ('shallow', 'uninspired'), ('uninspired', 'plot'), ('plot', 'fails'), ('

In [ ]:
# Positive bigram count
for bigram in positive_bigrams:
    bigram_count = positive_bigrams.count(bigram)
    print(f"{bigram}: {bigram_count}")

('forrest', 'gump'): 1
('gump', 'absolute'): 1
('absolute', 'masterpiece'): 1
('masterpiece', 'tom'): 1
('tom', 'hank'): 1
('hank', 'delivers'): 1
('delivers', 'unforgettable'): 1
('unforgettable', 'performance'): 1
('performance', 'storytelling'): 1
('storytelling', 'heartwarming'): 1
('heartwarming', 'movie'): 1
('movie', 'journey'): 1
('journey', 'life'): 1
('life', 'make'): 1
('make', 'laugh'): 1
('laugh', 'cry'): 1
('cry', 'appreciate'): 1
('appreciate', 'simple'): 1
('simple', 'beauty'): 1
('beauty', 'existence'): 1
('existence', 'shawshank'): 1
('shawshank', 'redemption'): 1
('redemption', 'timeless'): 1
('timeless', 'classic'): 1
('classic', 'powerful'): 1
('powerful', 'theme'): 1
('theme', 'hope'): 1
('hope', 'friendship'): 1
('friendship', 'redemption'): 1
('redemption', 'make'): 1
('make', 'mustwatch'): 1
('mustwatch', 'morgan'): 1
('morgan', 'freeman'): 1
('freeman', 'tim'): 1
('tim', 'robbins'): 1
('robbins', 'give'): 1
('give', 'exceptional'): 1
('exceptional', 'performan

In [ ]:
# Negative bigram count
for bigram in negative_bigrams:
    bigram_count = negative_bigrams.count(bigram)
    print(f"{bigram}: {bigram_count}")

('last', 'airbender'): 1
('airbender', 'disaster'): 1
('disaster', 'film'): 1
('film', 'adaptation'): 1
('adaptation', 'butcher'): 1
('butcher', 'beloved'): 1
('beloved', 'animated'): 1
('animated', 'series'): 1
('series', 'wooden'): 1
('wooden', 'act'): 1
('act', 'convolute'): 1
('convolute', 'storytelling'): 1
('storytelling', 'cringeworthy'): 1
('cringeworthy', 'special'): 1
('special', 'effect'): 3
('effect', 'letdown'): 1
('letdown', 'fan'): 1
('fan', 'newcomer'): 1
('newcomer', 'alike'): 1
('alike', 'another'): 1
('another', 'transformer'): 1
('transformer', 'movie'): 1
('movie', 'mindless'): 1
('mindless', 'explosion'): 1
('explosion', 'incoherent'): 1
('incoherent', 'plotlines'): 1
('plotlines', 'overreliance'): 1
('overreliance', 'cgi'): 1
('cgi', 'franchise'): 1
('franchise', 'desperately'): 1
('desperately', 'need'): 1
('need', 'overhaul'): 1
('overhaul', 'emoji'): 1
('emoji', 'movie'): 1
('movie', 'blatant'): 1
('blatant', 'cash'): 1
('cash', 'grab'): 1
('grab', 'shallow'):

In [ ]:
# Function to calculate bigram probabilities
def calculate_bigram_probabilities(bigrams,unigrams):

    # Calculate conditional probabilities
    bigram_probabilities = {}
    for bigram in bigrams:
        word1, word2 = bigram
        bigram_count = bigrams.count(bigram)
        unigram_count = unigrams.count(word1)
        bigram_probabilities[bigram] = (bigram_count) / (unigram_count)

    return bigram_probabilities

In [ ]:
# Positive bigram probabilities
positive_bigram_probabilities = calculate_bigram_probabilities(positive_bigrams,final_positive_words)
print(positive_bigram_probabilities)


{('forrest', 'gump'): 1.0, ('gump', 'absolute'): 1.0, ('absolute', 'masterpiece'): 1.0, ('masterpiece', 'tom'): 0.3333333333333333, ('tom', 'hank'): 1.0, ('hank', 'delivers'): 1.0, ('delivers', 'unforgettable'): 1.0, ('unforgettable', 'performance'): 1.0, ('performance', 'storytelling'): 0.5, ('storytelling', 'heartwarming'): 1.0, ('heartwarming', 'movie'): 1.0, ('movie', 'journey'): 1.0, ('journey', 'life'): 0.5, ('life', 'make'): 1.0, ('make', 'laugh'): 0.2, ('laugh', 'cry'): 1.0, ('cry', 'appreciate'): 1.0, ('appreciate', 'simple'): 1.0, ('simple', 'beauty'): 1.0, ('beauty', 'existence'): 1.0, ('existence', 'shawshank'): 1.0, ('shawshank', 'redemption'): 1.0, ('redemption', 'timeless'): 0.5, ('timeless', 'classic'): 0.5, ('classic', 'powerful'): 0.5, ('powerful', 'theme'): 1.0, ('theme', 'hope'): 1.0, ('hope', 'friendship'): 1.0, ('friendship', 'redemption'): 1.0, ('redemption', 'make'): 0.5, ('make', 'mustwatch'): 0.2, ('mustwatch', 'morgan'): 1.0, ('morgan', 'freeman'): 1.0, ('fre

In [ ]:
# Negative bigram probabilities
negative_bigram_probabilities = calculate_bigram_probabilities(negative_bigrams,final_negative_words)
print(negative_bigram_probabilities)

{('last', 'airbender'): 1.0, ('airbender', 'disaster'): 1.0, ('disaster', 'film'): 0.5, ('film', 'adaptation'): 0.14285714285714285, ('adaptation', 'butcher'): 0.5, ('butcher', 'beloved'): 1.0, ('beloved', 'animated'): 1.0, ('animated', 'series'): 1.0, ('series', 'wooden'): 1.0, ('wooden', 'act'): 1.0, ('act', 'convolute'): 0.3333333333333333, ('convolute', 'storytelling'): 0.5, ('storytelling', 'cringeworthy'): 1.0, ('cringeworthy', 'special'): 0.5, ('special', 'effect'): 1.0, ('effect', 'letdown'): 0.3333333333333333, ('letdown', 'fan'): 1.0, ('fan', 'newcomer'): 0.5, ('newcomer', 'alike'): 1.0, ('alike', 'another'): 1.0, ('another', 'transformer'): 1.0, ('transformer', 'movie'): 1.0, ('movie', 'mindless'): 0.25, ('mindless', 'explosion'): 1.0, ('explosion', 'incoherent'): 1.0, ('incoherent', 'plotlines'): 1.0, ('plotlines', 'overreliance'): 1.0, ('overreliance', 'cgi'): 1.0, ('cgi', 'franchise'): 1.0, ('franchise', 'desperately'): 1.0, ('desperately', 'need'): 1.0, ('need', 'overhau

In [ ]:
def classify_movie_review(test_review):
    N = 2

    # Remove punctuation and stop words, tokenize and lemmatize the words in the test_review
    test_review_no_punct = ''.join([char for char in test_review if char not in string.punctuation])
    test_words = word_tokenize(test_review_no_punct)
    test_words = lemmatize_reviews(test_words)

    # Generate bigrams for the test review
    test_bigrams = list(bigrams(test_words))

    # Calculate the N-gram probability for the test review
    positive_prob = 1.0
    negative_prob = 1.0

    # Calculate probabilities for positive bigrams
    for bigram in test_bigrams:
        if bigram in positive_bigram_probabilities:
            positive_prob *= positive_bigram_probabilities[bigram]

    # Calculate probabilities for negative bigrams
    for bigram in test_bigrams:
        if bigram in negative_bigram_probabilities:
            negative_prob *= negative_bigram_probabilities[bigram]

    # Predict the category of the test movie review
    if positive_prob > negative_prob:
       prediction_explanation = "The review is positive."
    elif positive_prob < negative_prob:
       prediction_explanation = "The review is negative."
    else:
       prediction_explanation = "The review is neutral."

    print(f"Positive Probability: {positive_prob}")
    print(f"Negative Probability: {negative_prob}")

    return prediction_explanation


##Calculate the Bi-Gram probability for the test movie review

In [ ]:
# Call the function and print the output
test_review = "It's clear that the movie has both its enthusiasts and critics. While it may not be to everyone's taste, it's worth watching with an open mind to form your own opinion."

output = classify_movie_review(test_review)
print(output)

Positive Probability: 1.0
Negative Probability: 1.0
The review is neutral.
